In [ ]:
def trade_on_prediction(model, LOB3Data, feature_columns, num_bins, window_len, delay_steps, lambda_, batch_size, device):
    # Convert the model to evaluation mode
    model.eval()

    # Initialize trading variables
    cash = 0
    assets = 0

    # Prepare the data
    data = calculate_lob_features(LOB3Data, num_bins, window_len, delay_steps, lambda_)

    categorical_features = ['current_imbalance_idx', 'current_price_movement']
    _, test_loader = get_dataloaders(data, feature_columns, categorical_features, batch_size=batch_size, val_data=True)

    # Loop over all the data in the test_loader
    with torch.no_grad():
        for X, _ in test_loader:
            X = X.to(device)
            predictions = model(X)
            predicted_classes = torch.argmax(predictions, axis=1)

            for predicted_class in predicted_classes:
                if predicted_class == 0:  # Predicted downward price movement
                    if assets < 0:  # If we have a short position
                        cash = cash + LOB3Data['MOAsk']  # Buy back
                        assets = assets + 1
                    else:
                        cash = cash + LOB3Data['MOBid']  # Sell
                        assets = assets - 1
                elif predicted_class == 2:  # Predicted upward price movement
                    if assets > 0:  # If we have a long position
                        cash = cash - LOB3Data['MOBid']  # Sell off
                        assets = assets - 1
                    else:
                        cash = cash - LOB3Data['MOAsk']  # Buy
                        assets = assets + 1

    # At the end of trading, liquidate position
    if assets > 0:
        cash = cash + assets * LOB3Data['MOBid'][-1]  # Sell off
    elif assets < 0:
        cash = cash + assets * LOB3Data['MOAsk'][-1]  # Buy back

    return cash
